In [1]:
import qsharp
import json
from diskcache import Cache
import re_utils
from math import ceil

re_utils.DEBUG = False
cache = Cache("~/quant-arith-cache/re-adders-out-of-pace")
qsharp.init(project_root="../")

@cache.memoize()
def estimate_resources_out_of_place_adder(op, n):
    if op == "QuantumArithmetic.WBC2023.AddWithOp":
        radix = ceil(n / 2)
        est = qsharp.estimate(f"EstimateUtils.RunRadix({n},{radix},{op},Std.Arithmetic.RippleCarryCGAddLE)")
    else:
        est = qsharp.estimate(f"EstimateUtils.Run3WayOp({n},{n},{n},{op})")
    return json.dumps(est)    
  
# These all are out-of-place adders modulo 2^n.
ops_and_max_n = [
  ("Std.Arithmetic.RippleCarryCGAddLE", "Gidney", 2**20),
  ("Std.Arithmetic.LookAheadDKRSAddLE", "DKRS", 2**20),
  ("QuantumArithmetic.CT2002.Add", "CT", 2**20),
  ("QuantumArithmetic.SC2023.Add_Mod2N", "LingStruct", 110218),
  ("QuantumArithmetic.GKDKH2021.Add_Mod2N", "Gayathri", 2**20),
  ("QuantumArithmetic.WLLQW2016.Add_Mod2N", "Wang", 2**20),      
  ("QuantumArithmetic.WBC2023.AddWithOp", "HiRadix", 2**20),
]

re_utils.run_re_experiments(
    ops_and_max_n, 
    estimate_resources_out_of_place_adder,
    title='(b) Out-of-place addition',
    special_markers={
        "Std.Arithmetic.RippleCarryCGAddLE":"1",
        "QuantumArithmetic.CT2002.Add":"2",
        "QuantumArithmetic.GKDKH2021.Add_Mod2N":"3",
        "QuantumArithmetic.WBC2023.AddWithOp":"4",
    } 
)

Charts saved to img/b_out_of_place_addition.png


In [2]:
re_utils.show_re_table(
    ops_and_max_n, 
    estimate_resources_out_of_place_adder, 2**10)

n=1024


,Algorithm,Logical qubits,Physical qubits,Logical depth,"Runtime, seconds"
0,Gidney,6302,4712284,3069,0.023324
1,DKRS,8334,6269148,13163,0.100039
2,CT,8377,6318194,12288,0.093389
3,LingStruct,24692,22048344,57771,0.485276
4,Gayathri,6302,4712284,3069,0.023324
5,Wang,6302,4712284,3069,0.023324
6,HiRadix,8372,6260584,11244,0.085454


In [3]:
re_utils.trendline_analysis(
    ops_and_max_n, 
    estimate_resources_out_of_place_adder)

,Algorithm,Logical qubits,Physical qubits,Logical depth,"Runtime, seconds"
0,Gidney,6.4859e+00 * n^0.9935,1.7846e+03 * n^1.1309,2.9816e+00 * n^1.0005,1.2422e-05 * n^1.0798
1,DKRS,8.3826e+00 * n^0.9962,2.6776e+03 * n^1.1307,1.2354e+01 * n^1.0045,5.6700e-05 * n^1.0803
2,CT,8.5723e+00 * n^0.9942,2.8613e+03 * n^1.1241,1.2000e+01 * n^1.0000,5.5880e-05 * n^1.0742
3,LingStruct,2.3844e+01 * n^1.0010,9.2385e+03 * n^1.1336,5.3014e+01 * n^1.0073,2.7850e-04 * n^1.0796
4,Gayathri,6.4859e+00 * n^0.9935,1.7846e+03 * n^1.1309,2.9816e+00 * n^1.0005,1.2422e-05 * n^1.0798
5,Wang,6.4859e+00 * n^0.9935,1.7846e+03 * n^1.1309,2.9816e+00 * n^1.0005,1.2422e-05 * n^1.0798
6,HiRadix,8.5493e+00 * n^0.9944,2.6313e+03 * n^1.1316,1.0890e+01 * n^1.0009,4.9941e-05 * n^1.0765
